## This is aim to visualize the crop progress in the Urbana and other town
Combined with the weather data and soil data. Interploate the weather data to create the CGDD,CGDD in the first and last three weeks. Also for CRPT



__Goodfield__  & __DeKalb__  from UIUC
and __Winnebago__ from the first.


In [78]:
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import os
import math
import re
import time
import pandas as pd
import numpy as np
import os
import warnings
import seaborn
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as dt
from datetime import datetime, timedelta
import seaborn as sns, numpy as np


In [79]:

def NA_count(arg1):
    col_list = arg1.columns
    i=0
    df = pd.DataFrame(columns = ["var", "missing_percent"])
    for col in col_list:
        
        a = len(arg1[col][pd.isnull(arg1[col])])/len(arg1)
        df.loc[i,"var"]= col
        df.loc[i,"missing_percent"]= a*100
        i= i+1
    return(df)

In [80]:
# read in the data for weather and both 
file_path= '/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/data'
filecsv_list = []
for root, dirs , files in os.walk(file_path):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            filecsv_list.append(file)
filecsv_list

['FieldSiteLocations.csv',
 'Farms_Soils_Year_LL.csv',
 'soil_dic_6L.csv',
 '2011_2017_ALL_FIRST_DATA.csv',
 '2011_2017_UIUC_ALL_DATA.csv',
 'soil_dic_interpolated.csv',
 'InterpolatedParameters.csv',
 'standardized_soil_fixed_utf8.csv']

In [81]:
Soil_city = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*Farms.*', f)][0]))
Soil_dic_6L = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*6L.*', f)][0]))
Soil_dic_6L.drop('Unnamed: 0', axis=1, inplace=True)
soil_dic_interpolated = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*dic.*', f)][0]))
soil_dic_interpolated.drop('Unnamed: 0', axis=1, inplace=True)

which is interesting that the soil data is unique for each. Direct match the FIRST or UIUC data with the soil data.

In [82]:
soil_dic_interpolated.head()

,Soil Name,Soil Desc,1_low,1_up,1_% Total Clay < 0.002,1_%Total Silt 0.002-0.05,1_% Total Sand 0.05-2,2_low,2_up,2_% Total Clay < 0.002,...,5_low,5_up,5_% Total Clay < 0.002,5_%Total Silt 0.002-0.05,5_% Total Sand 0.05-2,6_low,6_up,6_% Total Clay < 0.002,6_%Total Silt 0.002-0.05,6_% Total Sand 0.05-2
0,Mokena,NaN,0,10,24.0,NaN,20.5,10,20,24.0,...,40,50,33.0,NaN,29.0,50,60,NaN,NaN,NaN
1,Myrtle,NaN,0,10,24.0,NaN,4.0,10,20,24.0,...,40,50,NaN,NaN,NaN,50,60,NaN,NaN,NaN
2,Gilford,"Coarse-loamy, mixed, superactive, mesic Typic ...",0,10,12.0,23.0,63.0,10,20,12.4,...,40,50,14.0,29.2,55.8,50,60,14.0,30.0,55.0
3,ParrA,"A Fine-loamy, mixed, active, mesic Oxyaquic Ar...",0,10,14.0,33.0,52.0,10,20,14.0,...,40,50,NaN,NaN,NaN,50,60,NaN,NaN,NaN
4,Raub,"Fine-silty, mixed, superactive, mesic Aquic Ar...",0,10,23.0,73.0,3.0,10,20,23.0,...,40,50,29.2,65.8,3.4,50,60,31.8,62.8,4.0


In [83]:
os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*UIUC.*', f)][0])
UIUC_2011_2017_ALL_DATA= pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*UIUC.*', f)][0]))

In [84]:
Soil_list = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*fixed.*', f)][0]))
UIUC_2011_2017_ALL_DATA= pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*UIUC.*', f)][0]))
UIUC_2011_2017_ALL_DATA['Location Town'].value_counts()
#

Monmouth      755
New Berlin    755
Mt. Morris    719
Goodfield     710
DeKalb        617
Erie          616
Perry         556
Urbana        528
St. Peter     435
Dwight        387
Belleville    350
Elkville      236
Fenton        103
Name: Location Town, dtype: int64

In [85]:
uiuc_soil_list= UIUC_2011_2017_ALL_DATA['Soil Type'].value_counts().index.values

In [86]:
Weather_interp= pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*InterpolatedParameters.*', f)][0]))
Weather_interp.head()
# Weather_interp['CITY'].value_counts()

,CITY,DATE,Interpolated TMIN,Interpolated TMAX,Interpolated PRCP,Interpolated AWND
0,F_BELLEVILLE,1/1/2011,28.0,59.0,0.006049,NaN
1,F_BELLEVILLE,1/2/2011,18.0,34.0,0.000000,NaN
2,F_BELLEVILLE,1/3/2011,22.0,45.0,0.000000,NaN
3,F_BELLEVILLE,1/4/2011,23.0,42.0,0.000000,NaN
4,F_BELLEVILLE,1/5/2011,20.0,41.0,0.000000,NaN


In [87]:
First_11_17_all= pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*\FIRST.*', f)][0]))
First_11_17_all.columns

Index(['DataSource', 'Company', 'Product', 'Technology', 'RM', 'Yield (Bu/A)',
       'Moisture %', 'Lodging %', 'Stand (x1000)', 'Gross $/A', 'Income Rank',
       'Year', 'Location_Town', 'Location_County', 'Location State', 'Season',
       'Prev_Crop', 'Previous_Pest', 'Previous Yield (bu/A)',
       'Previous Yield (yrs)', 'Soil_Type', 'Soil_slope', 'Soil_drained',
       'Soil_Irr', 'Soil condition', 'Soil_P', 'Soil_K', 'Soil_pH', 'Soil_OM%',
       'Soil_CEC', 'PestMang', 'FertN (units)', 'FertP (units)',
       'FertK (units)', 'Seed_date', 'Seed rate (/A)', 'Seed space (in)',
       'Harvest_date', 'Harvest rate (/A)'],
      dtype='object')

In [88]:
First_11_17_all['Gross $/A']

0         699.0
1        1137.5
2        1161.5
3        1274.3
4        1293.0
5        1351.0
6        1204.0
7         799.0
8         716.0
9         724.0
10        760.0
11        787.0
12       1158.7
13       1211.7
14       1211.8
15       1218.1
16       1161.9
17       1228.2
18        795.0
19        811.0
20        809.0
21        782.0
22        761.0
23        880.0
24        810.0
25        786.0
26        721.0
27        719.0
28        725.0
29        899.0
          ...  
12284    1228.0
12285     736.0
12286     727.0
12287     734.0
12288    1033.0
12289    1082.0
12290    1022.0
12291     764.0
12292     770.0
12293     835.0
12294    1338.0
12295    1084.0
12296    1192.0
12297     806.0
12298     792.0
12299     802.0
12300     809.0
12301     807.0
12302     789.0
12303     806.0
12304     772.0
12305     798.0
12306     834.0
12307     941.0
12308     819.0
12309     858.0
12310     819.0
12311     858.0
12312     752.0
12313     696.0
Name: Gross $/A, dtype: 

In [89]:
First_11_17_all.columns ==UIUC_2011_2017_ALL_DATA.columns
#len(First_11_17_all.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True, False, False,
        True,  True, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False])

In [90]:
len(UIUC_2011_2017_ALL_DATA.columns)
(UIUC_2011_2017_ALL_DATA.columns)

Index(['DataSource', 'Company', 'Product', 'Technology', 'RM', 'Yield (Bu/A)',
       'Moisture %', 'Lodging %', 'Stand (x1000)', 'Gross $/A', 'Income Rank',
       'Year', 'Location Town', 'Location County', 'Location State', 'Season',
       'Previous Crop', 'Previous Pest', 'Previous Yield (bu/A)',
       'Previous Yield (yrs)', 'Soil Type', 'Soil Slope', 'Soil Drained',
       'Soil Irr', 'Soil Condition', 'Soil_P', 'Soil_K', 'Soil_pH', 'Soil_OM%',
       'Soil_CEC', 'PestMang', 'FertN (units)', 'FertP (units)',
       'FertK (units)', 'Seed Date', 'Seed Rate (/A)', 'Seed Space (in)',
       'Harvest Date', 'Harvest Rate (/A)'],
      dtype='object')

In [91]:
First_11_17_all.columns  = UIUC_2011_2017_ALL_DATA.columns

__The columns len and rough contents matched for uiuc and first data sets. Reset the column names of both sets according to the uiuc sets.__

In [92]:
UIUC_2011_2017_ALL_DATA.head()

,DataSource,Company,Product,Technology,RM,Yield (Bu/A),Moisture %,Lodging %,Stand (x1000),Gross $/A,...,Soil_CEC,PestMang,FertN (units),FertP (units),FertK (units),Seed Date,Seed Rate (/A),Seed Space (in),Harvest Date,Harvest Rate (/A)
0,UIUC,AgriGold,A6442STXRIB,GMO,107.0,223.0,14.9,0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
1,UIUC,AgriGold,A6472VT3PRIB,GMO,109.0,230.0,15.7,0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
2,UIUC,AgriGold,A6499STXRIB,GMO,110.0,248.0,18.3,0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
3,UIUC,AgriGold,A6533VT3PRIB,GMO,112.0,249.0,17.3,0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
4,UIUC,AgriGold,A6559STXRIB,GMO,113.0,247.0,16.8,0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN


In [93]:
UIUC_2011_2017_ALL_DATA.groupby('Year')['Moisture %'].describe()

Year       
2011  count    1134.000000
      mean       20.797795
      std         3.765813
      min         0.000000
      25%        17.825000
      50%        21.400000
      75%        23.575000
      max        30.600000
2012  count     628.000000
      mean       18.540287
      std         3.284182
      min        12.800000
      25%        16.000000
      50%        17.800000
      75%        21.000000
      max        30.000000
2013  count    1123.000000
      mean       22.322262
      std         3.061640
      min        15.500000
      25%        19.900000
      50%        22.000000
      75%        24.500000
      max        31.600000
2014  count    1177.000000
      mean       19.399320
      std         3.170884
      min        13.700000
      25%        17.100000
      50%        18.800000
      75%        21.300000
      max        32.000000
2015  count     986.000000
      mean       16.410041
      std         2.744444
      min        11.900000
      25%       

To avoid too much missing, drop those with 10% missing var

In [94]:
tmp = NA_count(UIUC_2011_2017_ALL_DATA)
UIUC_2011_2017_ALL_DATA = UIUC_2011_2017_ALL_DATA.loc[:,(tmp['missing_percent']<30).values]
tmp 

,var,missing_percent
0,DataSource,0
1,Company,0
2,Product,0.0443328
3,Technology,0
4,RM,0.0443328
5,Yield (Bu/A),0.650214
6,Moisture %,0.635437
7,Lodging %,0
8,Stand (x1000),100
9,Gross $/A,100


In [95]:
NA_count(First_11_17_all)

,var,missing_percent
0,DataSource,0
1,Company,4.97807
2,Product,4.98619
3,Technology,4.97807
4,RM,5.0268
5,Yield (Bu/A),4.97807
6,Moisture %,4.97807
7,Lodging %,4.97807
8,Stand (x1000),4.97807
9,Gross $/A,4.97807


In [96]:
UIUC_2011_2017_ALL_DATA.groupby('Location Town')['Seed Date'].value_counts()

Location Town  Seed Date
Belleville     5/5/2015      81
               5/5/2014      71
               4/25/2013     70
               4/18/2016     69
               5/18/2017     59
DeKalb         5/7/2014     121
               5/8/2013     108
               5/8/2017     103
               5/8/2011     101
               4/24/2012     97
               4/23/2016     87
Dwight         5/5/2011     118
               5/6/2014      96
               4/19/2017     86
               4/19/2016     79
               4/24/2014      8
Elkville       5/18/2011     85
               5/1/2015      81
               4/25/2013     70
Erie           5/7/2014     121
               5/8/2013     108
               4/23/2015    102
               5/3/2011     101
               4/24/2012     97
Fenton         5/8/2017     103
Goodfield      5/6/2011     118
               4/17/2015    110
               5/8/2013     110
               5/8/2014     104
               4/18/2012    103
               

In [97]:
UIUC_2011_2017_ALL_DATA.groupby('Location Town')['Soil Type'].value_counts()

Location Town  Soil Type               
Belleville     Ebbert silt loam            291
               Caseyville silt loam         59
DeKalb         Flanagan silty clay loam    514
               Elpaso silty clay loam      103
Dwight         Reddick silty clay loam     387
Elkville       Okaw silt loam              236
Erie           Beaucoup silty clay loam    616
Fenton         Coffeen silt loam           103
Goodfield      Ipava silt loam             710
Monmouth       Sable silty clay loam       755
Mt. Morris     Muscatine silt loam         719
New Berlin     Sable silt loam             755
Perry          Herrick silt loam           556
St. Peter      Bluford silt loam           376
               Darmstadt silt loam          59
Urbana         Flanagan silt loam          528
Name: Soil Type, dtype: int64

In [98]:
Soil_list.head()

,Num_of_Layers,Soil Name,Soil Desc,Pedon No.,1_Layer,1_Horizon,1_low,1_up,1_Lab Texture,"1_CEC Activity, CEC7/Clay, Weighted Average",...,30_Water Content 33 kPa,30_Water Content 1500 kPa,30_% OC (WB),30_NH4OAC Extractable Bases Ca,30_NH4OAC Extractable Bases Mg,30_NH4OAC Extractable Bases Na,30_NH4OAC Extractable Bases K,30_NH4OAC Extractable Bases Sum Bases,30_CEC8 Sum Cats,30_CEC7 NH4 OAC
0,5,Mokena,NaN,NaN,NaN,H1,0,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,Myrtle,NaN,NaN,NaN,A,0,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,Gilford,"Coarse-loamy, mixed, superactive, mesic Typic ...",75IL103004,IL24067,Ap,0,10,sl,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,ParrA,"A Fine-loamy, mixed, active, mesic Oxyaquic Ar...",78IL103015,IL25822,Ap,0,10,l,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Raub,"Fine-silty, mixed, superactive, mesic Aquic Ar...",61IL019002,IL18852,A1,0,10,SIL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:

def get_temperature(CITY):
    ind = Weather_interp['CITY'].apply(lambda x:  x.split('_')[1].lower().capitalize()) == CITY.lower().capitalize()

    temperature =Weather_interp.loc[ind,:]

    temperature = temperature[['DATE','CITY','Interpolated TMIN','Interpolated TMAX']]

    temperature = temperature.dropna()
    temperature = temperature.drop_duplicates()
    temperature.columns = ['DATE', 'CITY', 'TMIN', 'TMAX']
    temperature.head()

    upper_index = (temperature[temperature['TMAX'] > 86].index.values.tolist())
    lower_index_tmin = (temperature[temperature['TMIN'] < 50].index.values.tolist())
    lower_index_tmax = (temperature[temperature['TMAX'] < 50].index.values.tolist())
    temperature.loc[upper_index, 'TMAX'] = 86
    temperature.loc[lower_index_tmin, 'TMIN'] = 50
    temperature.loc[lower_index_tmax, 'TMAX'] = 50
    temperature ['GDD'] = temperature.mean(axis=1) - 50
    temperature['PRCP']  = Weather_interp.loc[ind,'Interpolated PRCP']
    return(temperature)
def  std_GDD (x):
    start_date=pd.to_datetime(x[0])
    end_date=pd.to_datetime(x[1])
    df = temperature
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index('DATE')
    df = df.sort()
    df = df.drop_duplicates()
    df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
    tmp = df['GDD'].std()#
    return(tmp)

def std_PRCP(x ):
    start_date=pd.to_datetime(x[0])
    end_date=pd.to_datetime(x[1])
    df = temperature
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index('DATE')
    df = df.sort()
    df = df.drop_duplicates()
    df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
    tmp = df['PRCP'].std()#
    return(tmp)




def row_cumulative_GDD(x,first = False ,last = False,length =21):
    if first == True:
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[0])+timedelta(days=length)
    elif last == True:
        start_date=pd.to_datetime(x[1])-timedelta(days=length)
        end_date=pd.to_datetime(x[1])
    else:
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[1])
        

    df = temperature
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index('DATE')
    df = df.sort()
    df = df.drop_duplicates()
    df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
    tmp = df['GDD'].sum()#
    return(tmp)

def row_cumulative_PRCP(x,first = False ,last = False,length =21):
    
    if first == True:
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[0])+timedelta(days=length)
    elif last == True:
        start_date=pd.to_datetime(x[1])-timedelta(days=length)
        end_date=pd.to_datetime(x[1])
    else:
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[1])

    df = temperature
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index('DATE')
    df = df.sort()
    df = df.drop_duplicates()
    df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
    tmp = df['PRCP'].sum()#
    return(tmp)


def get_df_UIUC(Urbana):
    temperature = get_temperature(Urbana) 
    Urbana_df = UIUC_2011_2017_ALL_DATA.loc[(UIUC_2011_2017_ALL_DATA['Location Town']== Urbana),:]
    Num_col = list(Urbana_df.select_dtypes([np.number]).columns.values)
    Num_col.extend(['Seed Date','Harvest Date','Soil Type'])
    Urbana_df = Urbana_df.loc[:,Num_col]
    Urbana_df['Soil Name'] = Urbana_df['Soil Type'].apply(lambda x: x.split()[0])
    Urbana_df = pd.merge(Urbana_df, Soil_dic_6L, how='left',on = 'Soil Name')




    # Urbana_df = Urbana_df.drop(['Soil Name', 'Soil Type'], axis=1)

    Urbana_df['CGDD'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_GDD,axis=1)
    Urbana_df['std_GDD'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(std_GDD,axis=1)
    Urbana_df['CPRCP']= Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_PRCP,axis=1)
    Urbana_df['std_PRCP']= Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_PRCP,axis=1)







    Urbana_df['CGDD_first21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,first = True),axis=1)
    Urbana_df['CGDD_last21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,last = True),axis=1)
    Urbana_df['CPRCP_first21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,first = True),axis=1)
    Urbana_df['CPRCP_last21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,last = True),axis=1)
 #   tmp = NA_count(Urbana )
#    Urbana  = Urbana .loc[:,(tmp['missing_percent']<10).values]
    return(Urbana_df)
#['Product','Year','Yield (Bu/A)','Seed Date','Harvest Date','Moisture %', 'Seed Rate (/A)','Lodging %']
#Urbana  =Urbana[col_new]


#ind = Weather_interp['CITY'].apply(lambda x:  x.split('_')[1].lower().capitalize()) == CITY.lower().capitalize()

In [111]:
Urbana  = get_df_UIUC('Urbana') 

Urbana.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Urbana.csv')
Urbana.describe()

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



,RM,Yield (Bu/A),Moisture %,Lodging %,Year,FertN (units),Seed Rate (/A),1_low,1_up,1_% Total Clay < 0.002,...,6_%Total Silt 0.002-0.05,6_% Total Sand 0.05-2,CGDD,std_GDD,CPRCP,std_PRCP,CGDD_first21,CGDD_last21,CPRCP_first21,CPRCP_last21
count,528.000000,528.000000,528.000000,528.000000,528.000000,528.000000,528.000000,528.0,528.0,528.0,...,528.0,528.0,528.000000,528.000000,528.000000,528.000000,528.000000,528.000000,528.000000,528.000000
mean,111.700758,221.892045,17.612121,0.484848,2013.818182,207.765152,34928.030303,0.0,10.0,19.5,...,59.7,3.0,2718.727272,6.778716,20.538405,20.538405,282.658129,379.011617,3.845622,2.755647
std,2.151174,31.695924,2.854053,1.914629,1.966776,4.169756,1208.968032,0.0,0.0,0.0,...,0.0,0.0,776.676974,0.778861,9.684675,9.684675,67.499055,130.728140,1.517908,1.450183
min,105.000000,141.000000,12.500000,0.000000,2011.000000,200.000000,34000.000000,0.0,10.0,19.5,...,59.7,3.0,1290.127060,5.375485,8.402227,8.402227,184.592503,138.035299,1.660745,0.549992
25%,110.000000,206.750000,15.200000,0.000000,2013.000000,210.000000,34000.000000,0.0,10.0,19.5,...,59.7,3.0,2925.631358,6.781281,13.042633,13.042633,235.161906,379.511141,3.640199,2.027796
50%,112.000000,232.000000,17.600000,0.000000,2014.000000,210.000000,34000.000000,0.0,10.0,19.5,...,59.7,3.0,3150.444740,6.904932,19.601301,19.601301,259.142905,394.665267,3.653800,2.810769
75%,113.000000,245.000000,19.400000,0.000000,2015.000000,210.000000,36500.000000,0.0,10.0,19.5,...,59.7,3.0,3170.954892,7.185606,30.372990,30.372990,330.150527,449.569914,4.216406,3.949940
max,117.000000,281.000000,27.900000,28.000000,2017.000000,210.000000,36500.000000,0.0,10.0,19.5,...,59.7,3.0,3315.178221,7.665619,32.609464,32.609464,378.110028,524.915638,6.625256,4.569842


In [112]:
DeKalb = get_df_UIUC('DeKalb') 

DeKalb.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/DeKalb.csv')
DeKalb.describe()


/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



,RM,Yield (Bu/A),Moisture %,Lodging %,Year,FertN (units),Seed Rate (/A),1_low,1_up,1_% Total Clay < 0.002,...,6_%Total Silt 0.002-0.05,6_% Total Sand 0.05-2,CGDD,std_GDD,CPRCP,std_PRCP,CGDD_first21,CGDD_last21,CPRCP_first21,CPRCP_last21
count,616.000000,602.000000,602.000000,617.000000,617.000000,617.000000,617.000000,514.0,514.0,514.0,...,5.140000e+02,514.0,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000
mean,108.961039,227.828904,21.089203,0.260940,2013.802269,186.904376,34769.854133,0.0,10.0,19.5,...,5.970000e+01,3.0,3291.946319,7.125741,21.260158,21.260158,313.853114,242.958996,3.436720,2.513391
std,2.911448,29.162845,3.592421,1.143153,2.076412,14.977654,1155.042085,0.0,0.0,0.0,...,1.422470e-14,0.0,132.597087,0.561093,6.019275,6.019275,41.905193,63.522998,0.798409,1.700728
min,99.000000,136.000000,13.600000,0.000000,2011.000000,160.000000,34000.000000,0.0,10.0,19.5,...,5.970000e+01,3.0,3099.892204,6.396917,15.514558,15.514558,216.240766,152.047388,2.058745,1.498833
25%,107.000000,213.000000,18.300000,0.000000,2012.000000,180.000000,34000.000000,0.0,10.0,19.5,...,5.970000e+01,3.0,3201.208235,6.421856,15.575970,15.575970,308.620725,181.076532,2.776998,1.553805
50%,109.000000,233.500000,21.000000,0.000000,2014.000000,200.000000,34000.000000,0.0,10.0,19.5,...,5.970000e+01,3.0,3294.755604,7.559103,17.712513,17.712513,329.136538,259.620150,3.627166,1.667684
75%,111.000000,249.000000,23.600000,0.000000,2016.000000,200.000000,36500.000000,0.0,10.0,19.5,...,5.970000e+01,3.0,3362.505557,7.652537,26.636887,26.636887,341.600756,288.081629,4.222765,2.496231
max,116.000000,288.000000,30.800000,17.000000,2017.000000,200.000000,36500.000000,0.0,10.0,19.5,...,5.970000e+01,3.0,3523.733553,7.710566,31.049290,31.049290,348.124531,340.583768,4.346228,6.238042


In [113]:
Monmouth = get_df_UIUC('Monmouth')  
Monmouth = Monmouth.loc[:,(tmp['missing_percent']<10).values]
Monmouth.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Monmouth.csv')
Monmouth.describe()

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



,RM,Yield (Bu/A),Moisture %,Lodging %,Year,FertN (units),Seed Rate (/A),1_low,1_up,1_% Total Clay < 0.002,...,6_%Total Silt 0.002-0.05,6_% Total Sand 0.05-2,CGDD,std_GDD,CPRCP,std_PRCP,CGDD_first21,CGDD_last21,CPRCP_first21,CPRCP_last21
count,755.000000,755.00000,755.000000,755.000000,755.000000,755.000000,755.000000,755.0,755.0,755.0,...,755.0,755.0,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000
mean,111.736424,233.69404,19.793907,0.627815,2013.782781,173.139073,34930.829139,0.0,10.0,30.0,...,63.0,2.0,2986.411182,6.679371,21.685712,21.685712,269.651401,306.718268,2.640914,3.171399
std,2.200358,18.87205,2.975125,1.885452,1.923047,4.643868,1209.747077,0.0,0.0,0.0,...,0.0,0.0,413.679545,1.131503,6.960226,6.960226,63.813299,154.251342,1.406030,1.978568
min,105.000000,130.00000,13.500000,0.000000,2011.000000,170.000000,34000.000000,0.0,10.0,30.0,...,63.0,2.0,2062.549824,4.215045,14.202806,14.202806,203.590609,0.000000,1.638644,0.000000
25%,110.000000,221.00000,17.600000,0.000000,2012.000000,170.000000,34000.000000,0.0,10.0,30.0,...,63.0,2.0,3089.436228,6.896145,15.162911,15.162911,212.135781,266.976302,1.715477,0.769447
50%,112.000000,233.00000,19.400000,0.000000,2014.000000,170.000000,34000.000000,0.0,10.0,30.0,...,63.0,2.0,3132.428042,6.950961,17.889132,17.889132,261.641169,381.150410,2.228880,4.105591
75%,113.000000,247.00000,22.000000,0.000000,2015.000000,180.000000,36500.000000,0.0,10.0,30.0,...,63.0,2.0,3185.459594,7.243642,30.422988,30.422988,270.646156,405.069317,3.031831,4.779034
max,117.000000,291.00000,27.800000,17.000000,2017.000000,180.000000,36523.000000,0.0,10.0,30.0,...,63.0,2.0,3326.174553,7.838302,31.660108,31.660108,404.161062,478.448299,6.455165,5.711260


In [114]:
Goodfield = get_df_UIUC('Goodfield') 
tmp = NA_count(Goodfield )
Goodfield = Goodfield.loc[:,(tmp['missing_percent']<10).values]
Goodfield.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Goodfield.csv')
Goodfield.describe()

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



,RM,Yield (Bu/A),Moisture %,Lodging %,Year,FertN (units),Seed Rate (/A),1_low,1_up,1_% Total Clay < 0.002,...,6_%Total Silt 0.002-0.05,6_% Total Sand 0.05-2,CGDD,std_GDD,CPRCP,std_PRCP,CGDD_first21,CGDD_last21,CPRCP_first21,CPRCP_last21
count,710.000000,710.000000,710.000000,710.000000,710.000000,710.000000,710.000000,710.0,710.0,710.0,...,710.0,710.0,710.000000,710.000000,710.000000,710.000000,710.000000,710.000000,710.000000,710.000000
mean,111.698592,248.580282,20.663239,0.366197,2013.797183,209.405634,34968.309859,0.0,10.0,25.0,...,54.0,1.0,3172.330383,6.862621,21.915320,21.915320,278.297828,353.769681,3.276863,3.607918
std,2.157252,23.720247,3.104120,1.664696,1.971158,7.856360,1218.705278,0.0,0.0,0.0,...,0.0,0.0,123.259992,0.432579,6.462494,6.462494,53.973350,73.983817,1.309861,1.266929
min,105.000000,186.000000,12.900000,0.000000,2011.000000,196.000000,34000.000000,0.0,10.0,25.0,...,54.0,1.0,2977.862657,5.954486,14.992186,14.992186,196.583844,240.022496,1.660745,1.412181
25%,110.000000,234.250000,18.400000,0.000000,2012.000000,200.000000,34000.000000,0.0,10.0,25.0,...,54.0,1.0,3086.297973,6.744128,15.434347,15.434347,235.161906,271.968974,2.228880,2.509930
50%,112.000000,254.000000,20.600000,0.000000,2014.000000,210.000000,34000.000000,0.0,10.0,25.0,...,54.0,1.0,3188.687175,6.950583,19.982639,19.982639,262.623081,373.656218,3.247079,4.267770
75%,113.000000,266.000000,23.000000,0.000000,2015.000000,218.000000,36500.000000,0.0,10.0,25.0,...,54.0,1.0,3198.965398,7.082082,30.343241,30.343241,341.600756,395.077820,4.227234,4.785277
max,117.000000,295.000000,30.000000,28.000000,2017.000000,220.000000,36500.000000,0.0,10.0,25.0,...,54.0,1.0,3362.181674,7.396456,30.460474,30.460474,347.631182,458.468037,6.118613,5.060320


In [ ]:
Urbana.groupby('Year')['Seed Date'].first().tolist()

In [ ]:
NA_count(Urbana)

In [ ]:
#set the seed date harvest date
Urbana.groupby('Year')['Seed Date'].value_counts(sort=True)
Seed_date = Urbana.groupby('Year')['Seed Date'].first().tolist()
Harvest_date = Urbana.groupby('Year')['Harvest Date'].first().tolist()
Year = Urbana['Year'].unique().tolist()
Year.sort()
Year
#Urbana.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Urbana.csv')

In [ ]:
NA_count(Urbana)

In [ ]:
Urbana.groupby('Year')['Harvest Date'].value_counts(sort=True)

In [ ]:
#distribution

plt.figure(figsize=(25,25))
sns.pairplot(Urbana, diag_kind="kde",hue="Year")
plt.title('Feature correlation plot for Urbana')
plt.show()


In [ ]:
# basic see have the yield varied

plt.figure(figsize=(15,15))
ax = sns.boxplot(y='Yield (Bu/A)', x="Year", hue="Year",data=Urbana)
plt.title('Yield for different year in Urbana')
plt.show()


In [ ]:
#visualize the correlation matrix

plt.figure(figsize=(40,40))
corr = Urbana.select_dtypes(include=['float64']).corr()
corr
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap,  center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Heated map for feature correlation plot for Urbana')
plt.show()

In [ ]:
def show_cumulative_GDD(start_date, end_date,city ):
    df = temperature
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index('DATE')
    df = df.sort()
    df = df.drop_duplicates()
    df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
    plt.scatter(y = df['GDD'].cumsum(), x = range(len(df['GDD'])), s = 20, label =  pd.to_datetime(Seed_date[i]).year)
    plt.title('Cumulative GDD of '+city+'\n'+'From '+str(start_date)+' to ' +str(end_date))#
    plt.xlabel('Number of Days')
    plt.ylabel('Cumulative GDD')

plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_GDD(start_date=Seed_date[i] , end_date=Harvest_date[i] ,city = 'Urbana')    
plt.legend()
plt.show()

In [ ]:
def show_cumulative_PRCP(start_date, end_date,city):
    df = temperature
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index('DATE')
    df = df.sort()
    df = df.drop_duplicates()
    df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
    plt.scatter(y = df['PRCP'].cumsum(), x = range(len(df['PRCP'])), s = 20, label =  pd.to_datetime(Seed_date[i]).year)
    plt.plot( np.arange(len(df['PRCP'])),  df['PRCP'].cumsum(), linestyle='-')
    plt.title('Cumulative PRCP of '+city+'\n'+'From '+str(start_date)+' to ' +str(end_date))#
    plt.xlabel('Number of Days')
    plt.ylabel('Cumulative PRCP')

plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_PRCP(start_date=Seed_date[i] , end_date=Harvest_date[i] ,city='Urbana')    
plt.legend( Year)
plt.show()

In [ ]:
# model

# head for Goodfield

In [ ]:

#distribution

plt.figure(figsize=(25,25))
sns.pairplot(Goodfield, diag_kind="kde",hue="Year")
plt.title('Feature distribution plot for Goodfield')
plt.show()

In [ ]:
# basic see have the yield varied

plt.figure(figsize=(15,15))
ax = sns.boxplot(y='Yield (Bu/A)', x="Year", hue="Year",data=Goodfield)
plt.title('Yield for different year in Goodfield')
plt.show()


In [ ]:
#visualize the correlation matrix

plt.figure(figsize=(40,40))
corr =Goodfield.select_dtypes(include=['float64']).corr()
corr
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap,  center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Heated map for feature correlation plot for Goodfield')
plt.show()

In [ ]:
corr

In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_GDD(start_date=Seed_date[i] , end_date=Harvest_date[i],city = 'Goodfield' )    
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_PRCP(start_date=Seed_date[i] , end_date=Harvest_date[i] ,city = 'Goodfield')    
plt.legend( Year)
plt.show()

## Check the freq of each towns

In [ ]:
    col = ['Product','Year','Yield (Bu/A)','Previous Yield (bu/A)','Seed_date', 'Harvest_date','Moisture %','Lodging %', 'Harvest Rate (/A)','Seed Rate (/A)','Seed Space (in)','Soil_Type' ]
    Winnebago_df =First_11_17_all.loc[First_11_17_all['Location_Town']=='Winnebago',col]
    Winnebago_df['Soil_Type'].value_counts()
    First_11_17_all['Location_Town'].value_counts()

__Check the type of soil in UIUC and First__

In [ ]:
first_soil_list= First_11_17_all['Soil Type'].value_counts().index.values
data_soil_list = np.append(uiuc_soil_list,first_soil_list)

In [ ]:
data_soil_list = np.unique(data_soil_list)
np.unique(data_soil_list)

In [115]:
def get_df_FIRST(Winnebago):
  
    temperature = get_temperature(Winnebago) 
    Winnebago_df =First_11_17_all.loc[First_11_17_all['Location Town']== Winnebago,:]
    Num_col = list(Winnebago_df.select_dtypes([np.number]).columns.values)
    Num_col.extend(['Seed Date','Harvest Date','Soil Type'])
    Winnebago_df = Winnebago_df.loc[:,Num_col]
    Winnebago_df['Soil Name'] = Winnebago_df['Soil Type'].apply(lambda x : x.split('/')[0] \
                                .split(' ')[0]).apply(lambda x : 'Muscatune' if x == 'Muscatine' else x)

    Winnebago_df = pd.merge(Winnebago_df, soil_dic_interpolated, how='left',on = 'Soil Name')

    Winnebago_df['CGDD'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_GDD,axis=1)
    Winnebago_df['std_GDD'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(std_GDD,axis=1)
    Winnebago_df['CPRCP']= Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_PRCP,axis=1)
    Winnebago_df['CGDD_first21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,first = True),axis=1)
    Winnebago_df['std_PRCP']= Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(std_PRCP,axis=1)
    Winnebago_df['CGDD_last21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,last = True),axis=1)
    Winnebago_df['CPRCP_first21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,first = True),axis=1)
    Winnebago_df['CPRCP_last21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,last = True),axis=1)

    return(Winnebago_df)

In [116]:
 



Winnebago = get_df_FIRST('Winnebago')

Winnebago = Winnebago.drop('Seed Rate (/A)',axis=1)

Winnebago.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Winnebago.csv')

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:41: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



In [117]:
Malta= get_df_FIRST('Malta')
Malta = Malta.drop('Seed Rate (/A)',axis=1)

Malta.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Malta.csv')

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:41: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



In [118]:
Towanda = get_df_FIRST('Towanda')
Towanda = Towanda.drop('Seed Rate (/A)',axis=1)

Towanda.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Towanda.csv')

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:41: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



In [119]:
Mazon = get_df_FIRST('Mazon')
Mazon= Mazon.drop('Seed Rate (/A)',axis=1)

Mazon.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Mazon.csv')

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:65: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:29: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:86: FutureWarning:

sort(....) is deprecated, use sort_index(.....)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:41: FutureWarning:

sort(....) is deprecated, use sort_index(.....)



In [ ]:
Winnebago.columns
Winnebago.groupby('Year')['Seed date'].first().tolist()

In [ ]:
#set the seed date harvest date
Winnebago.groupby('Year')['Seed date'].value_counts(sort=True)
Seed_date = Winnebago.groupby('Year')['Seed date'].first().tolist()
Harvest_date = Winnebago.groupby('Year')['Harvest date'].first().tolist()
Year = Winnebago['Year'].unique().tolist()
Year.sort()
Year
Winnebago.to_csv('/home/ddi/data/Yield_Prediction/yield_prediction_modeling/Kewei/town_csv/Winnebago.csv')

In [ ]:
#distribution

plt.figure(figsize=(25,25))
sns.pairplot(Winnebago, diag_kind="kde",hue="Year")
plt.title('Feature distribution plot for Goodfield')
plt.show()

In [ ]:
# basic see have the yield varied

plt.figure(figsize=(15,15))
ax = sns.boxplot(y='Yield (Bu/A)', x="Year", hue="Year",data=Winnebago)
plt.title('Yield for different year in Winnebago')
plt.show()


In [ ]:
#visualize the correlation matrix

plt.figure(figsize=(40,40))
corr = Winnebago.select_dtypes(include=['float64']).corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap,  center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Heated map for feature correlation plot for Winnebago')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_GDD(start_date=Seed_date[i] , end_date=Harvest_date[i],city = 'Winnebago' )    
plt.legend(Year)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_PRCP(start_date=Seed_date[i] , end_date=Harvest_date[i] ,city = 'Winnebago')    
plt.legend(Year)
plt.show()